In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


Importing Data from excel sheet and getting familiar with data with shape, head attributes etc

In [2]:
Data = pd.read_excel(r'C:\Users\z00456zb\Downloads\Assignment 3 (1)\Assignment 3\Dataset.xlsx',header=None)
print(Data.shape,Data.columns)
print(Data.head())
mylist =Data.stack().groupby(level=0).apply(list).tolist()

(7501, 20) Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
            19],
           dtype='int64')
              0          1           2                 3             4   \
0         shrimp    almonds     avocado    vegetables mix  green grapes   
1        burgers  meatballs        eggs               NaN           NaN   
2        chutney        NaN         NaN               NaN           NaN   
3         turkey    avocado         NaN               NaN           NaN   
4  mineral water       milk  energy bar  whole wheat rice     green tea   

                 5     6               7             8             9   \
0  whole weat flour  yams  cottage cheese  energy drink  tomato juice   
1               NaN   NaN             NaN           NaN           NaN   
2               NaN   NaN             NaN           NaN           NaN   
3               NaN   NaN             NaN           NaN           NaN   
4               NaN   NaN             NaN           

Getting all unique item

In [3]:
uniq_set=set()
for i in mylist:
    for j in i:
        uniq_set.add(j)
print(len(uniq_set))
unique_list=list(uniq_set)

119


To make use of the apriori module given by mlxtend library, we need to convert the dataset according to it’s format.
Apriori module requires a dataframe that has 0 and 1 data. 
The data we have is all string (name of items), we need to One Hot Encode the data i.e convert to 0s and 1s

In [5]:
#converting to one hot encode
itemset = set(unique_list)
encoded_vals = []
for index, row in Data.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]
encoded_df = pd.DataFrame(encoded_vals)

In [6]:
print(type(encoded_df),encoded_df.shape)
print(encoded_df.head())

<class 'pandas.core.frame.DataFrame'> (7501, 119)
   shampoo  escalope  bramble  rice  cider  nonfat milk  cookies  brownies  \
0        0         0        0     0      0            0        0         0   
1        0         0        0     0      0            0        0         0   
2        0         0        0     0      0            0        0         0   
3        0         0        0     0      0            0        0         0   
4        0         0        0     0      0            0        0         0   

   oatmeal  dessert wine  ...  low fat yogurt  green grapes  mineral water  \
0        0             0  ...               1             1              1   
1        0             0  ...               0             0              0   
2        0             0  ...               0             0              0   
3        0             0  ...               0             0              0   
4        0             0  ...               0             0              1   

   energy dr

For support 0.5% and 1% with different confidences range (2%,90%)

In [7]:
final_df=pd.DataFrame()
sup=0.005
for i in range (1,3):
    conf=.1
    print("Support ",str(sup*100)+" %")
    freq_items = apriori(encoded_df, min_support=sup, use_colnames=True)
    freq_items.sort_values(by=['support'],ascending=False)
    print(freq_items.shape)
    print(freq_items.head())
    sup+=0.005
    print("\nRules with confidence "+str(.02*100)+"%")
    rules = association_rules(freq_items, metric="confidence", min_threshold=.02)
    rules.sort_values(by=['lift'],ascending=False)
    print(rules.shape)
    print(rules.head())
    final_df=final_df.append(rules)
    for j in range (1,9):
        rules = association_rules(freq_items, metric="confidence", min_threshold=conf)
        rules.sort_values(by=['lift'],ascending=False)
        print("\nRules with confidence "+str(conf*100)+"%")
        print(rules.shape)
        print(rules.head())
        conf+=.1
        final_df=final_df.append(rules)
        if rules.shape==(0,9):
            break
        
    print("*********************\n")


Support  0.5 %
(725, 2)
    support       itemsets
0  0.079323     (escalope)
1  0.018797         (rice)
2  0.010532        (cider)
3  0.010399  (nonfat milk)
4  0.080389      (cookies)

Rules with confidence 2.0%
(1940, 9)
              antecedents consequents  antecedent support  consequent support  \
0                 (pasta)  (escalope)            0.015731            0.079323   
1              (escalope)     (pasta)            0.079323            0.015731   
2              (tomatoes)  (escalope)            0.068391            0.079323   
3              (escalope)  (tomatoes)            0.079323            0.068391   
4  (mushroom cream sauce)  (escalope)            0.019064            0.079323   

    support  confidence      lift  leverage  conviction  
0  0.005866    0.372881  4.700812  0.004618    1.468107  
1  0.005866    0.073950  4.700812  0.004618    1.062867  
2  0.005466    0.079922  1.007555  0.000041    1.000651  
3  0.005466    0.068908  1.007555  0.000041    1.000555  

For support between (2%, 10% ) with support (2%,90%)

In [8]:
sup=0.01
for i in range (1,10):
    conf=.1
    sup+=.01
    print("Support ",str(sup*100)+" %")
    freq_items = apriori(encoded_df, min_support=sup, use_colnames=True)
    freq_items.sort_values(by=['support'],ascending=False)
    print(freq_items.shape)
    print(freq_items.head())
    print("\nRules with confidence "+str(.02*100)+"%")
    rules = association_rules(freq_items, metric="confidence", min_threshold=.02)
    rules.sort_values(by=['lift'],ascending=False)
    print(rules.shape)
    print(rules.head())
    final_df=final_df.append(rules)
    for j in range (1,9):
        rules = association_rules(freq_items, metric="confidence", min_threshold=conf)
        rules.sort_values(by=['lift'],ascending=False)
        print("\nRules with confidence "+str(conf*100)+"%")
        print(rules.shape)
        print(rules.head())
        final_df=final_df.append(rules)
        conf+=.1
        if rules.shape==(0,9):
            break
        
    print("*********************\n")


Support  2.0 %
(103, 2)
    support            itemsets
0  0.079323          (escalope)
1  0.080389           (cookies)
2  0.033729          (brownies)
3  0.068391          (tomatoes)
4  0.058526  (whole wheat rice)

Rules with confidence 2.0%
(100, 9)
       antecedents         consequents  antecedent support  \
0      (spaghetti)          (tomatoes)            0.174110   
1       (tomatoes)         (spaghetti)            0.068391   
2  (mineral water)          (tomatoes)            0.238368   
3       (tomatoes)     (mineral water)            0.068391   
4  (mineral water)  (whole wheat rice)            0.238368   

   consequent support   support  confidence      lift  leverage  conviction  
0            0.068391  0.020931    0.120214  1.757755  0.009023    1.058905  
1            0.174110  0.020931    0.306043  1.757755  0.009023    1.190117  
2            0.068391  0.024397    0.102349  1.496530  0.008095    1.037830  
3            0.238368  0.024397    0.356725  1.496530  0.00809

       antecedents      consequents  antecedent support  consequent support  \
0  (mineral water)      (chocolate)            0.238368            0.163845   
1      (chocolate)  (mineral water)            0.163845            0.238368   
2      (spaghetti)  (mineral water)            0.174110            0.238368   
3  (mineral water)      (spaghetti)            0.238368            0.174110   
4  (mineral water)           (eggs)            0.238368            0.179709   

    support  confidence      lift  leverage  conviction  
0  0.052660    0.220917  1.348332  0.013604    1.073256  
1  0.052660    0.321400  1.348332  0.013604    1.122357  
2  0.059725    0.343032  1.439085  0.018223    1.159314  
3  0.059725    0.250559  1.439085  0.018223    1.102008  
4  0.050927    0.213647  1.188845  0.008090    1.043158  

Rules with confidence 20.0%
(9, 9)
       antecedents      consequents  antecedent support  consequent support  \
0  (mineral water)      (chocolate)            0.238368       

In [9]:
#to check max support
freq_items = apriori(encoded_df, min_support=.2, use_colnames=True)
freq_items.sort_values(by=['support'],ascending=False)
print(freq_items)
print("\n")
freq_items = apriori(encoded_df, min_support=.3, use_colnames=True)
freq_items.sort_values(by=['support'],ascending=False)
print(freq_items)

    support         itemsets
0  0.238368  (mineral water)


Empty DataFrame
Columns: [support, itemsets]
Index: []


In [10]:
print(final_df.shape)

(5442, 9)


In [11]:
final_df.to_csv(r'DMAssignmentResult.csv')